In [57]:
# Available namespaces

from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, VOID, XMLNS, XSD

print(CSVW.seeAlso)
print(DC.seeAlso)
print(DCAT.seeAlso)
print(DCTERMS.seeAlso)
# print(DOAP.seeAlso)
# print(FOAF.seeAlso)
print(ODRL2.seeAlso)
print(ORG.seeAlso)
print(OWL.seeAlso)
print(PROF.seeAlso)
# print(PROV.seeAlso)
# print(RDF.seeAlso)
print(RDFS.seeAlso)
print(SDO.seeAlso)
print(SH.seeAlso)
# print(SKOS.seeAlso)
print(SOSA.seeAlso)
print(SSN.seeAlso)
print(TIME.seeAlso)
print(VOID.seeAlso)
# print(XMLNS.seeAlso)
print(XSD.seeAlso)

http://www.w3.org/ns/csvw#seeAlso
http://purl.org/dc/elements/1.1/seeAlso
http://www.w3.org/ns/dcat#seeAlso
http://purl.org/dc/terms/seeAlso
http://www.w3.org/ns/odrl/2/seeAlso
http://www.w3.org/ns/org#seeAlso
http://www.w3.org/2002/07/owl#seeAlso
http://www.w3.org/ns/dx/prof/seeAlso
http://www.w3.org/2000/01/rdf-schema#seeAlso
https://schema.org/seeAlso
http://www.w3.org/ns/shacl#seeAlso
http://www.w3.org/ns/ssn/seeAlso
http://www.w3.org/ns/sosa/seeAlso
http://www.w3.org/2006/time#seeAlso
http://rdfs.org/ns/void#seeAlso
http://www.w3.org/2001/XMLSchema#seeAlso


In [141]:
from dotenv import load_dotenv
load_dotenv()
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth

CLIENT_ID=os.getenv("CLIENT_ID")
CLIENT_SECRET=os.getenv("CLIENT_SECRET")
REDIRECT_URI="http://localhost:5566"
SCOPE = "user-library-read user-read-currently-playing streaming user-read-playback-state user-read-recently-played"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

In [142]:
%%capture
saved_tracks = sp.current_user_saved_tracks(50)
for idx, item in enumerate(saved_tracks['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])


In [101]:
def ms_to_iso8064_duration(time_in_ms):
    time_in_s = round(time_in_ms / 1000, 2)
    return f'PT{time_in_s}S'

Schema mapping

```
-----
Track
-----
Context: https://schema.org/
Type: MusicRecording
album: MusicRecording:inAlbum
artist: MusicRecording:byArtist
duration: MusicRecording:duration (ISO 8601)

------
Artist
------
@context: https://schema.org/
@type: MusicGroup
name: "Kendrick Lamar"
track: [
    {
        @type: MusicRecording
        name: love
    }
]

-----
Album
-----
Context: https://schema.org/
Type: MusicAlbum
artist: MusicRecording:byArtist

```

| key | schema |
| - | - |
| played_at |  |
| album_name | |
| album_href | |
| album_id | |
| album_images | |
| album_release_date | |
| artists | |
| first_artist_href | |
| first_artist_id | |
| first_artist_name | |
| name | |
| popularity |  |
| href | subject [for track] |
| duration_ms | duration [in ISO 8601] |

In [137]:
from rdflib import Graph, URIRef, Literal
g = Graph()

last_50_tracks_played = sp.current_user_recently_played(50)
for idx, item in enumerate(last_50_tracks_played['items']):
    add_track_to_graph(g, item)

In [138]:
def add_track_to_graph(graph, item):
    track = item['track']
    track_href = URIRef(track['href'])
    track_name = Literal(track['name'])
    track_duration = Literal(ms_to_iso8064_duration(track['duration_ms']))
    track_played_at = Literal(item['played_at'])
    artist_href = URIRef(track['artists'][0]['href'])
    artist_name = Literal(track['artists'][0]['name'])
    album_href = URIRef(track['album']['href'])
    album_name = Literal(track['album']['name'])

    graph.add((track_href, RDF.type, SDO.MusicRecording))
    graph.add((track_href, FOAF.name, track_name))
    graph.add((track_href, SDO.duration, track_duration))
    graph.add((track_href, SDO.byArtist, artist_href))
    graph.add((track_href, SDO.inAlbum, album_href))
    graph.add((track_href, SDO.startTime, track_played_at))
    # graph.add((track_href, SDO.endTime, Literal(played_at))) # startTime + duration
    graph.add((artist_href, RDF.type, SDO.MusicGroup))
    graph.add((artist_href, FOAF.name, artist_name))
    graph.add((album_href, RDF.type, SDO.MusicAlbum))
    graph.add((album_href, FOAF.name, album_name))


print(g.serialize(format="turtle").decode("utf-8"))

Artist <https://api.spotify.com/v1/artists/2NArTWfXN317Ok1Ale3xkX> ;
    ns1:duration "PT403.82S" ;
    ns1:inAlbum <https://api.spotify.com/v1/albums/0wlICzORLGl8yJwnc8q74I> ;
    ns1:startTime "2021-02-05T18:16:37.407Z" .

<https://api.spotify.com/v1/tracks/4CfkxZ4w0qCNuSA0hMJPeH> a ns1:MusicRecording ;
    ns2:name "Rare Happiness" ;
    ns1:byArtist <https://api.spotify.com/v1/artists/6uElH4moADg7AGB3DCGOwy> ;
    ns1:duration "PT351.21S" ;
    ns1:inAlbum <https://api.spotify.com/v1/albums/3HELUMHz9zL4T3BnwrXiR1> ;
    ns1:startTime "2021-02-05T15:37:19.998Z" .

<https://api.spotify.com/v1/tracks/4F2r6VndVq8LC2ZoPPQMwP> a ns1:MusicRecording ;
    ns2:name "Super Solid - Radio Edit" ;
    ns1:byArtist <https://api.spotify.com/v1/artists/0cKckxKy6R6LfDXp7TroK8> ;
    ns1:duration "PT279.72S" ;
    ns1:inAlbum <https://api.spotify.com/v1/albums/2wMUKeM1ayYSGd47HNfqlU> ;
    ns1:startTime "2021-02-05T18:39:31.712Z" .

<https://api.spotify.com/v1/tracks/4Xr9UKLq3YGnExa50oLfAw> a ns1:Mu

In [139]:
import requests

# curl -X PUT -H "Content-Type: text/turtle" \
#   -d "<ex:s> <ex:p> <ex:o>." \
#   http://localhost:3000/myfile.ttl
def put_graph(graph, resource_name, address = 'http://localhost:3000/solid-spotify/'):
    headers = {"content-type": "text/turtle"}
    url = f'{address}{resource_name}'
    data = graph.serialize(format="turtle").decode("utf-8")

    response = requests.put(url, headers = headers, data = data)

print(response.text)

In [140]:
put_graph(g, 'recent_tracks-2.ttl')